In [147]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [214]:
Predicted = pd.read_csv("./Seurat/Outputs/output_Sim21.csv")
Target=pd.read_csv("./Data/Sim21/target.csv")

In [215]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)
Predicted.drop(columns="prediction.score.max",inplace=True)


In [216]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [217]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [218]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [219]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [220]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [221]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [222]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [223]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [224]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [225]:
np.mean(np.nan_to_num(d))

0.6814257079195042

In [226]:
d

array([0.57732741, 0.78693739, 0.47840145, 0.87301899, 0.87643662,
       0.6242692 , 0.82649088, 0.82836524, 0.56981083, 0.7903215 ,
       0.50019517, 0.65149755, 0.47546197])

## Jenson Shannon Divergence

In [227]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.4598845460155602

## F1 Score, Precision, Recall

In [228]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [229]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [230]:
d

,Precision,Recall,F1-score
0.100,0.900990,0.429825,0.582001
0.200,0.867987,0.455411,0.597388
0.300,0.690647,0.497409,0.578313
0.010,0.749889,0.537266,0.626016
0.001,0.722837,0.559150,0.630544


## Wasserstein Distance

In [231]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.05697945952009553

## KL Divergence

In [232]:
to_be_kept=(Predicted>0)

In [233]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [234]:
kl=scipy.special.rel_entr(Target,Predicted)

In [235]:
np.mean(kl)

0.04480631330969528